In [ ]:
# coding: utf-8

import openpyxl
import configparser
import zipfile,os
import datetime
import time
import sys

''''
configsetting
''' 
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')
iwb = config_ini.get('DEFAULT', 'InputWorkbook')
isn = config_ini.get('DEFAULT', 'InputSheetName')


'''
excellsetting
'''
inputWb = openpyxl.load_workbook(iwb,keep_vba=True)
inputWs = inputWb[ isn ]

'''
エクセルの値取得
'''
fileSizeLimit        = inputWs.cell(row = 20, column = 3).value
pictureFilePath      = inputWs.cell(row = 9,  column = 3).value
zipSaveFilePath      = inputWs.cell(row = 10, column = 3).value
folderLimtTotalSize  = inputWs.cell(row = 21, column = 3).value
executeInterval      = inputWs.cell(row = 18, column = 3).value
topZipFileName       = inputWs.cell(row = 16, column = 3).value

print(folderLimtTotalSize)
print(fileSizeLimit)
print(executeInterval)
print(pictureFilePath)
print(zipSaveFilePath)
print(topZipFileName)


In [ ]:



def get_dir_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total





In [ ]:


num = 1

while(True):
    #1～rangeまで出力
    print(str(num) +"回目の処理を開始します。")



    """
    写真サイズとフォルダサイズ比較
    """

    #バイトをメガバイトに変換する・
    pictureSize = get_dir_size('images') / 1024 / 1024
    print(str(pictureSize) + "MB")

    #写真サイズがフォルダサイズより大きい場合
    print("写真サイズと指定したフォルダサイズの比較をします。")
    if(pictureSize > folderLimtTotalSize):
        print("写真の容量が指定の容量を超えています。")
        print("処理を終了します。")
        sys.exit(1)

        #処理を終了する。
    else:
        print("処理を継続する。")
    

    
    """
    時刻の取得
    """
    #時刻の取得
    dateName        = datetime.datetime.now()
    topDateAddName  = dateName.strftime('%Y%m%d%H%M%S')
    

    """
    ZIPの数を比較
    """
    #ジップファイルの数を計算
    readZipCounter = 0
    for file in os.listdir(zipSaveFilePath):
        base, ext = os.path.splitext(file)
        if ext == '.zip':
            readZipCounter += 1
    print("ZIPファイル数"+ str(readZipCounter) + "個")    
    print("指定のZIPファイル上限数と現状作成完了しているZip数を比較します")

    #指定の上限数よりカウンター小さいときに処理を継続する。

    if(readZipCounter <= fileSizeLimit -1 ):
        #Zip化する。
        with zipfile.ZipFile(str(zipSaveFilePath) + '\\' + topZipFileName + topDateAddName + '.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:

            #file記載する。
            os.chdir(str(pictureFilePath))
                
            #jpgファイルを格納
            for file in os.listdir(pictureFilePath):
                pictureBase, ext = os.path.splitext(file)
                if ext == '.jpg':
    
                    zipf.write(  pictureBase + '.jpg', compress_type=zipfile.ZIP_DEFLATED)
            zipf.close()
            os.chdir(str("C:\python\work\pythonTools\FileZipper"))
            print("zipを作成しました。")
    else:
        print("zipファイルが上限に達しました。")
        break
    #インクリメント

    print(str(num) +"回目の処理を終了します。")
    num = num + 1
    print("次の実行まで待機中")
    time.sleep(executeInterval * 60)


        

In [ ]:
print("プログラムを終了します")